In [ ]:
!pip install nltk
!pip install scikit-learn
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.0 MB/s eta 0:00:00


In [ ]:
import PyPDF2
from nltk.corpus import stopwords
import nltk
import os
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

def extract_text(file_path):
    with open(file_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

def generate_dialogue_summary(text, num_sentences=4):
    # Tokenización de oraciones
    sentences = sent_tokenize(text)

    # Eliminación de palabras vacías (stopwords)
    stop_words = set(stopwords.words("spanish"))
    sentences = [sentence for sentence in sentences if sentence.strip() not in stop_words]

    # Cálculo de la matriz TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)

    # Obtención de las importancias de las frases
    sentence_importances = tfidf_matrix.sum(axis=1)

    # Ordenamiento de las frases por importancia
    ranked_sentences = sorted(((importance, sentence) for importance, sentence in zip(sentence_importances, sentences)), reverse=True)

    # Selección de las primeras num_sentences frases más relevantes
    top_sentences = [s[1] for s in ranked_sentences[:num_sentences]]

    # Crear diálogos combinando las frases seleccionadas
    dialogue = []
    for i in range(1, len(top_sentences), 2):
        dialogue.append(top_sentences[i-1] + " " + top_sentences[i])

    # Unir los diálogos en un solo texto
    dialogue_text = "\n".join(dialogue)

    return dialogue_text

# Extraer texto del PDF
pdf_path = os.environ.get("pdf_path", input("Ingrese la ruta del archivo"))  # Ruta al archivo PDF
output_file1 = input("Ingrese donde desea guardar el archivo")
output_file = output_file1 + "/resumen.txt"
articulo_texto = extract_text(pdf_path)
articulo_texto = articulo_texto.replace("[ edit ]", " ")

# Generar resumen en forma de diálogo a partir de la importancia de las frases
resumen_dialogo = generate_dialogue_summary(articulo_texto, num_sentences=4)

# Guardar el resumen traducido en un archivo
with open(output_file, "w") as file:
    file.write(resumen_dialogo)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Ingrese la ruta del archivo/content/CV.pdf
Ingrese donde desea guardar el archivo.


In [ ]:
import discord
from discord.ext import commands
import os
import PyPDF2
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

intents = discord.Intents.default()
intents.typing = False
intents.presences = False

bot = commands.Bot(command_prefix='!', intents=intents)

def extract_text(file_path):
    with open(file_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

def generate_dialogue_summary(text, num_sentences=4):
    # Tokenización de oraciones
    sentences = sent_tokenize(text)

    # Eliminación de palabras vacías (stopwords)
    stop_words = set(stopwords.words("spanish"))
    sentences = [sentence for sentence in sentences if sentence.strip() not in stop_words]

    # Cálculo de la matriz TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)

    # Obtención de las importancias de las frases
    sentence_importances = tfidf_matrix.sum(axis=1)

    # Ordenamiento de las frases por importancia
    ranked_sentences = sorted(((importance, sentence) for importance, sentence in zip(sentence_importances, sentences)), reverse=True)

    # Selección de las primeras num_sentences frases más relevantes
    top_sentences = [s[1] for s in ranked_sentences[:num_sentences]]

    # Crear diálogos combinando las frases seleccionadas
    dialogue = []
    for i in range(1, len(top_sentences), 2):
        dialogue.append(top_sentences[i-1] + " " + top_sentences[i])

    # Unir los diálogos en un solo texto
    dialogue_text = "\n".join(dialogue)

    return dialogue_text

@bot.event
async def on_ready():
    print(f'Logged in as {bot.user.name} - {bot.user.id}')

@bot.command(name='resumir')
async def resumir(ctx):
    await ctx.send("¡Hola! Sube el archivo PDF que deseas resumir.")

    def check(message):
        return message.author == ctx.author and message.attachments

    try:
        message = await bot.wait_for("message", check=check, timeout=60)
        attachment = message.attachments[0]
        if not attachment.filename.endswith('.pdf'):
            await ctx.send("El archivo adjunto debe ser un PDF.")
            return

        file_data = await attachment.read()
        pdf_path = os.environ.get("pdf_path", input("Ingrese la ruta del archivo"))  # Ruta al archivo PDF
        output_file1 = input("Ingrese donde desea guardar el archivo")
        output_file = output_file1 + "/resumen.txt"
        articulo_texto = extract_text(pdf_path)
        articulo_texto = articulo_texto.replace("[ edit ]", " ")

        # Generar resumen en forma de diálogo a partir de la importancia de las frases
        resumen_dialogo = generate_dialogue_summary(articulo_texto, num_sentences=4)

        # Guardar el resumen traducido en un archivo
        with open(output_file, "w") as file:
            file.write(resumen_dialogo)

    except TimeoutError:
        await ctx.send("Ha pasado mucho tiempo. Por favor, vuelve a ejecutar el comando y sube el archivo.")

TOKEN = os.environ.get("Your_Token")
bot.run(TOKEN)
